# Single camera calibration

## Single camera calibration with the knowledge of image's meta-information


An image taken by a camera gives the following information:<br>
- focus length = 25.7mm <br>
- image resolution (horizontal&vertical)= 350 PPI <br>
- image dimension = 5472(width) * 3648(height)<br>

Given this information, our goal is to calculate the camera calibtration matrix K
$P =
 \begin{pmatrix}
  f\cdot m_x & s & p_x \\
  0 &  f\cdot m_y & p_y  \\
  0 & 0 & 1  \\
 \end{pmatrix}$

Assumptions: 
1. pixels have no skew → s = 0
2. pixels are square → $f_x$ = $f_y$ = f
3. the principal point is at the
center of the image (measured in pixel)-> $p_x = \frac{1}{2} \cdot \text{image width in pixel}$
$p_y = \frac{1}{2} \cdot \text{image height in pixel}$

In [19]:
import numpy as np

def get_calibration_mat_K(r,c,f,mx,my):
  '''
  r: width of the image (pixel)
  c: height of the image (pixel)
  f: focus length (mm)
  mx,my: resolution in horizontal and vertical direction (pixel per mm)
  '''
  fmx,fmy = f*mx,f*my # focus length in pixel
  px,py = 0.5*r,0.5*c # principal point in pixel
  K = np.diag([fmx,fmy,1])
  K[0,2],K[1,2] = px,py
  return K

In [26]:
# for our particular example, the parameters are as follows
f = 25.7 # mm
mx = my = 350 # pixel/inch
# convert the unit pixel/inch to pixel/mm with the relation
# 1 ppi = 0.03937 ppm
mx = my = round(mx*0.03937,0) # pixel/mm
r,c = 5472, 3648 # image dimension in pixel
np.set_printoptions(suppress=True)
K = get_calibration_mat_K(r,c,f,mx,my)
print('Intrinsic camera parameters:','\n',K)

Intrinsic camera parameters: 
 [[ 359.8    0.  2736. ]
 [   0.   359.8 1824. ]
 [   0.     0.     1. ]]


## Single camera calibration with an image of an object with known geometry

- Take an image of an object with known geometry
- Given a set of corresponding 3D points and 2D points, we can use least squares to estimate projection matrix
$P =
 \begin{pmatrix}
  p_{11} & p_{12} & p_{13} & p_{14}\\
   p_{12} & p_{22} & p_{23} & p_{24}  \\
   p_{13} & p_{32} & p_{33} & p_{34}\\
 \end{pmatrix}$
- Intrinsic and extrinsic parameters can then be extracted from P
 1. Getting intrinsic matrix and rotation matrix with QR factorization
P = KR where \\
 $P = \begin{pmatrix}
  p_{11} & p_{12} & p_{13} \\
   p_{12} & p_{22} & p_{23} \\
   p_{13} & p_{32} & p_{33} \\
 \end{pmatrix}$,
 $K = \begin{pmatrix}
  f\cdot m_x & s & p_x \\
  0 &  f\cdot m_y & p_y  \\
  0 & 0 & 1  \\
 \end{pmatrix}$ and $
  R = \begin{pmatrix}
  r_{11} & r_{12} & r_{13} \\
   r_{12} & r_{22} & r_{23} \\
   r_{13} & r_{32} & r_{33} \\
 \end{pmatrix} $
 2. Getting translation vector T by solving
  $\begin{pmatrix}
  p_{14} \\
  p_{24} \\
  p_{34} \\
 \end{pmatrix}$ = K⋅$\begin{pmatrix}
  t_{x} \\
  t_{y} \\
  t_{z} \\
 \end{pmatrix}$

